In [1]:
import os
import pickle
import pandas as pd
import numpy as np

In [8]:
def generate_filename(year, month, type_file):
    if type_file.lower() == "inp":
        return f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
    if type_file.lower() == "out":
        directorio_out = "output"
        if not os.path.exists(f"{directorio_out}"):
            os.mkdir(f"{directorio_out}")
        return f"{directorio_out}/yellow_tripdata_{year:04d}-{month:02d}.parquet"

def read_data(filename, categorical, year, month):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    #generate ride_id
    df["ride_id"] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    return df


def prepare_data(df, categorical):
    return df[categorical].to_dict(orient='records')


def load_model():
    with open('model.bin', 'rb') as f_in:
        dv, model = pickle.load(f_in)

    return dv, model

In [3]:
def apply_model(df, categorical):
    X_dicts = prepare_data(df, categorical)
    dv_sl, model_sl = load_model()
    X_val = dv_sl.transform(X_dicts)
    y_pred = model_sl.predict(X_val)

    return y_pred


def calculate_statistics(y_predic):
    dict_statistics = dict()
    dict_statistics["std_y_predic"] = np.std(y_predic)
    
    return dict_statistics

def save_result(df, y_predic, filename):
    df_result = pd.DataFrame()
    df_result["ride_id"] = df["ride_id"]
    df_result["y_predic"] = y_predic
    
    df_result.to_parquet(filename, engine='pyarrow', compression=None, index=False)

In [6]:
def run():
    year = 2023
    month = 3
    CATEGORICAL = ['PULocationID', 'DOLocationID']
   
    input_file = generate_filename(year, month, "inp")
    output_file = generate_filename(year, month, "out")
    data_fram = read_data(input_file, CATEGORICAL, year, month)
   
    predictions = apply_model(data_fram, CATEGORICAL)
    print(calculate_statistics(predictions))

    save_result(data_fram, predictions, output_file)


In [7]:
run()

{'std_y_predic': 6.247488852238703}
